https://huggingface.co/docs/transformers/en/training

In [1]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 102.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/scierc")
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3219 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/974 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [4]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
from transformers import AutoModelForSequenceClassification

num_labels = len(set(train_data['labels']))
model = AutoModelForSequenceClassification.from_pretrained("allenai/cs_roberta_base", num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/cs_roberta_base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [7]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,  # <- follow paper for smaller dataset
    num_train_epochs=5,  # <- paper used 100 epoch for TAPT instead
    per_device_train_batch_size=128,  # <- follow paper for smaller dataset
    per_device_eval_batch_size=128,  # <- follow paper for smaller dataset
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    adam_beta1=0.9,  # <- follow paper
    adam_beta2=0.98,  # <- follow paper
    adam_epsilon=1e-6,  # <- follow paper
    lr_scheduler_type='constant',  # <- follow paper for smaller dataset
    weight_decay=0.01,  # <- follow paper
    warmup_ratio=0.06,  # <- follow paper
    logging_steps=100,  # <- follow paper
    output_dir="./training_output_2",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,0.724100,0.605456,0.810989,0.799939,0.837412,0.722378,0.810989,0.741958,0.813242
2,0.490400,0.636684,0.826374,0.837388,0.848475,0.755613,0.826374,0.724143,0.800769
3,0.363700,0.543204,0.848352,0.823899,0.846526,0.773438,0.848352,0.794585,0.843941
4,0.236700,0.647025,0.848352,0.815020,0.862296,0.827349,0.848352,0.812954,0.850289
5,0.229600,0.731225,0.846154,0.829847,0.861647,0.770768,0.846154,0.779140,0.843393


TrainOutput(global_step=1010, training_loss=0.4662259562180774, metrics={'train_runtime': 1572.5096, 'train_samples_per_second': 10.235, 'train_steps_per_second': 0.642, 'total_flos': 4234962547584000.0, 'train_loss': 0.4662259562180774, 'epoch': 5.0})

In [11]:
trainer.evaluate()

{'eval_loss': 0.7312249541282654,
 'eval_accuracy': 0.8461538461538461,
 'eval_precision_macro': 0.8298469254467707,
 'eval_precision_weighted': 0.8616465548623601,
 'eval_recall_macro': 0.7707681781094677,
 'eval_recall_weighted': 0.8461538461538461,
 'eval_f1_macro': 0.7791400530954282,
 'eval_f1_weighted': 0.8433925281389785,
 'eval_runtime': 14.0981,
 'eval_samples_per_second': 32.274,
 'eval_steps_per_second': 0.284,
 'epoch': 5.0}

In [12]:
trainer.predict(tokenized_test_data)

PredictionOutput(predictions=array([[-1.0045216 , -2.1435614 ,  6.7120476 , ...,  0.17718646,
        -0.33469817, -3.4080117 ],
       [-0.9588018 , -1.7830284 ,  6.828723  , ...,  0.25980645,
        -0.65423125, -3.0644007 ],
       [ 0.6000477 , -2.6183176 ,  6.0158854 , ...,  1.0110248 ,
        -0.24087171, -3.1674464 ],
       ...,
       [-0.7300279 ,  2.5135593 , -1.7484708 , ..., -2.5430162 ,
         6.3290534 , -1.447252  ],
       [-0.33625695, -0.97531927, -2.1535573 , ..., -1.2671269 ,
        -1.2205673 , -1.6276095 ],
       [ 1.7427027 ,  6.834685  , -2.2999547 , ..., -2.1208928 ,
        -0.53253865, -1.3697338 ]], dtype=float32), label_ids=array([2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 2, 5, 2, 0, 0, 0, 0, 2, 0, 0,
       0, 5, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 2,
       0, 0, 4, 0, 0, 4, 4, 1, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 5, 5, 5, 0,
       6, 5, 6, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 5, 5, 0, 0, 0, 5, 5, 0, 0,
       5, 0, 6, 6, 6, 0, 0, 0, 0, 0, 4, 